# [Bonus Tasks](svm_segm.ipynb)

In [11]:
import numpy as np
import matplotlib.pyplot as plt

import scipy.ndimage as ndi
import skimage.morphology as morph
import skimage.util
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [12]:
def blend_result(img, seg, border_radius=1, interior_opacity=1, interior_opacity_decay=0.9, color=(0,1,0)):
    img  = np.dstack([img] * 3).copy()
    img -= img.min()
    img /= img.max()
    selem  = morph.disk(border_radius)
    seg_bd = np.logical_xor(morph.binary_dilation(seg, selem), morph.binary_erosion(seg, selem))
    mask_decay = ndi.distance_transform_edt(seg)
    for i in range(3):
        opacity = interior_opacity / pow(1 + mask_decay[seg], interior_opacity_decay)
        img[:,:,i][seg] = color[i] * opacity + (1 - opacity) * img[:,:,i][seg]
        img[:,:,i][seg_bd] = color[i]
    return img

In [13]:
patch_sizes = (32, 32)

---

**[Task 1.1.]()** Implement `create_data_matrix`:

In [14]:
from skimage.util import view_as_blocks

def create_data_matrix(img, patch_sizes):

    blocks = view_as_blocks(img, block_shape=patch_sizes) #Bild in Blöcke zerlegen 

    num_blocks_y, num_blocks_x = blocks.shape[:2]# Anzahl Patches berechnen

    X = blocks.reshape(num_blocks_y * num_blocks_x, patch_sizes[0]* patch_sizes[1])

    return X



**[Task 1.2.]()** Implement `create_gt_labels_vector`:

In [15]:
def create_gt_labels_vector(gt_image, patch_sizes):

    blocks = view_as_blocks(gt_image, block_shape=patch_sizes)
    num_blocks_y, num_blocks_x = blocks.shape[:2]

    # prozent Vordergrunf pro Patch berechnen
    patch_area = patch_sizes[0] * patch_sizes[1]

    # Summe der Pixel (0/1) = Anzahl Vordergrund Pixel
    fg_fraction = blocks.reshape(num_blocs_y * num_blocks_x, patch_area).mean(axis=1)

    # Label Zuweisung
    y = np.zeros_like(fg_fraction, dtype=int)
    y[fg_fraction > 0.5]=1
    y[fg_fraction == 0]= -1

    return y


**[Task 1.3.]()** Create the SVM classifier:

In [16]:
clf = make_pipeline(StandardScaler(), SVC(class_weight='balanced', gamma=0.1))

**[Task 1.3 (a).]()** Create the data matrices for the images `dna-33` and `dna-44`:

In [17]:
img1 = plt.imread('data/NIH3T3/im/dna-33.png')
if img1.ndim == 3:
    img1 = img1.mean(axis=2)

img2 = plt.imread('data/NIH3T3/im/dna-44.png')
if img2.ndim == 3:
    img2 = img2.mean(axis=2)

# Dartenmatrix erstellen
X1 = create_data_matrix(img1, patch_sizes)
X2 = create_data_matrix(img2, patch_sizes)



**[Task 1.3 (b).]()** Create the corresponding ground truth label vectors:

In [18]:
gt1 = plt.imread('data/NIH3T3/gt/33.png')
if gt1.ndim == 3:
    gt1 = gt1.mean(axis=2)


gt2 = plt.imread('data/NIH3T3/gt/44.png')
if gt2.ndim == 3:
    gt2 = gt2.mean(axis=2)

# Dartenmatrix erstellen
y1 = create_data_matrix(gt1, patch_sizes)
y2 = create_data_matrix(gt2, patch_sizes)

if X1.ndim == 1:
    X1 = X1.reshape(1,-1)

if X2.ndim == 1:
    X2 = X2.reshape(1,-1)

y1 = y1.flatten()
y2 = y2.flatten()


**[Task 1.3 (c).]()** Create the *combined* data matrices and ground truth label vectors:

In [19]:
# maske für Label !=0
mask1 = (y1 != 0)
mask2 = (y2 !=0)

# nur relevante patches auswählen 
X1_sel = X1[mask1]
X2_sel = X2[mask2]

y1_sel = y1[mask1]
y2_sel = y2[mask2]

# zusammenführen
X_train = np.concatenate([X1_sel, X2_sel], axis=0)
y_train = np.concatenate([y1_sel, y2_sel], axis=0)

print('shape X_train', X_train.shape)
print('shape y_train', y_train.shape)
print('labels', np.unique(y_train))# nur -1 und 1

IndexError: boolean index did not match indexed array along dimension 0; dimension is 1344 but corresponding boolean dimension is 1376256

**[Task 1.3 (d).]()** Train the classifier using the data matrix and label vectors from above:

In [ ]:
clf.fit(X_train, y_train)
print()

ValueError: Expected 2D array, got 1D array instead:
array=[0.10588235 0.10588235 0.10980392 ... 0.16078432 0.14901961 0.14901961].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

**[Task 1.4.]()** Implement the function `predict_image`:

Test your implementation:

In [ ]:
img = plt.imread(f'data/NIH3T3/im/dna-0.png')
seg = predict_image(img)

plt.figure()
plt.imshow(blend_result(img, seg))

NameError: name 'predict_image' is not defined

**[Task 1.5.]()** Perform batch processing: